In [ ]:
from OSCC_postprocessing.cine.functions_videos import *
from pathlib import Path
import json

# Video is in uint12 
video_bits = 12
brightness_levels = 2.0**video_bits



file = Path(r"G:\OSCC\LubeOil\BC20241003_HZ_Nozzle1\cine\T1\2.cine")
# Load the .cine file into a 3D numpy array (gray scale, shape: (Frame, Height, Width))
video = load_cine_video(file, frame_limit=100)

# Normalize the grayscale video to [0, 1] brightness range
video = video / brightness_levels

json_file = Path(r"G:\OSCC\LubeOil\BC20241003_HZ_Nozzle1\cine\T1\config.json")
with open(json_file, 'r', encoding='utf-8') as f:
    # Load metadata
    data = json.load(f)
    number_of_plumes = int(data['plumes'])
    # offset = float(data['offset']) # Not used in multi hole (Calculated later by FFT)
    centre = (float(data['centre_x']), float(data['centre_y']))
    ir_ = float(data["inner_radius"])   # inner radius (Injector radius)
    or_ = float(data["outer_radius"])   # outer radius (Quatz window radius)



print(f"The injector has {number_of_plumes} plumes.")
print(f"The nozzle is centred at ({centre[0]:.2f}, {centre[1]:.2f}) in image coordinates.")